    This notebook contains a small analysis of the dataset we are working with.

In [27]:
import pandas as pd

#### Reading the dataset

In [53]:
df = pd.read_csv("data.tsv",sep="\t")

#### Details about the Dataset

In [54]:
schema_df = pd.read_csv('schema.csv', index_col='column name')
schema_df

,description
column name,
Name,Name of the person
Selected,Whether the person is selected to work in the ...
Date of Birth,Date of birth
Age,How old the person is
Gender,Male or Female
LinkedIn Profile,link to a linkedin profile
University,Name of the university
Other,Name of School (if included)
Degree Course,Name of the Course


In [55]:
df['Selected'].value_counts()

No     860
Yes     34
Name: Selected, dtype: int64

#### Cleaning the Dataset for preprocessing

- The column named `Unnamed: 22` is removed
- Longer Columns Names are reduced to short names
- Changing inputs from the `Selected` column to `True` or `False` 
- Rounding up the Age

In [56]:
# removing the a column for the dataframe
del df['Unnamed: 22']
# changing the names of the columns
df.rename(columns={'What do you think is your life’s purpose and why do you think having a purpose is important? (100 words or less)':'Purpose','Describe 5 things that attracted you to Unilever.':'About company'}, inplace=True)
# changing all the Yes and No to True or False
df['Selected'] = df['Selected'].map({'Yes':True,'No':False})
# rounding off the age
df['Age'] = df['Age'].apply(round)

In [51]:
df

,Name,Selected,Date of Birth,Age,Gender,LinkedIn Profile,University,Degree Course,Area Of Study,Graduation Year,...,Professional Qualification,Clubs & Associations,Sports,Leadership,Skills & Abilities,Working Experience,Function,Source of Information,purpose,about_company
0,Ambepitiyage Nalin Imesh De Silva,Yes,12/7/1996,24.15,Male,Yes,CA Sri Lanka,ACCA,Finance and Accountancy,2019,...,No,1) Interact Club of Isipathana College - Assis...,Swimming (2011/12),NaN,- Ability to handle work under pressure while ...,No,Customer Development,Word of mouth,The purpose of my life would be to take every ...,- Industry - Branding - Exposure - Recognition...
1,Dakshika Chandalee Wijesekera,Yes,8/2/1996,24.50,Female,Yes,University of Peradeniya,Bachelors degree in Psychology (Hons),Psychology,2021,...,None.,"SCHOOL #Interact Club- President, Director of...","#Athletics (400m, 800m, 1500m & relays) #Gymna...",SCHOOL #School Captain & House Captain -Scho...,Always look at the bigger picture Through tra...,Yes,HR,University,My life purpose is living a successful life by...,Being a reputed and a multi national company. ...
2,Supeshala Colombathanthri,Yes,5/2/1997,23.75,Female,Yes,SLIIT,Batchelors in Business Management,Specialization in Business Analytics,2021,...,CIMA student,"•Gavels club , SLIIT (Editor) •AIESEC , SLIIT ...","•Tennis (National Ranker, rank 08) •Wushu (Nat...","•School Prefect (Senior, Juniorand Primary) •S...","Self Decipline it, is the main skill that I le...",Yes,Marketing,Word of mouth,"My purpose is to, ""Inspire the Vibe to, beat t...",01)Company reputation 02)Brands of the company...
3,Deshan Anselam Benhur,Yes,7/12/1995,25.55,Male,Yes,"Swinburne University of Technology, Melbourne,...",Completed my Bachelor's degree,Mechanical Engineering,2019,...,Graduate Engineer membership- Board of Enginee...,"• President, Engineers Australia, Swinburne Sa...",Basketball,School Senior Prefect University Student council,• Project Management - As the organizing chair...,Yes,Supply Chain (Engineering Students are Preferred),Facebook,My life's purpose is to genuinely serve and he...,Unilever has State of the art factories and as...
4,Masinghe Arachchige Yasaransi Pabasara Masinghe,Yes,2/13/1995,25.97,Female,Yes,University of Peradeniya,Bachelor of Business Administration,Human Resource Management,2020,...,CIMA Adv. Dip MA and almost have completed the...,Vice President(External Relations)- Blue Marbl...,Participated inter house hockey matches at sch...,Vice President (External Relations) Blue Marbl...,Acquiring a skill may probably be facile but p...,No,HR,Word of mouth,To be a little spark in everyone's never wande...,At the very first visit of mine to Unilever du...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889,Rathnayake Mudiyanselage Tashikala Pavithra Ra...,No,6/14/1995,25.63,Female,Yes,University of Sabaragamuwa,Hons in ICT,ICT,2021,...,Followed English course,No,"Volleyball, Netball","Prefect,president in Buddhist society,member a...",Working as a L.CPL in cadet team I was made as...,No,Customer Development,Instagram,Having a true purpose about life will only all...,1. A place where people from all cultures are ...
890,Chrishani christy,No,10/27/1999,21.26,Female,Yes,Bms,BSC in internation business management,Management and marketing,2021,...,AAT,I’m the secretary of the student council in Bm...,Basket ball,Secretary of student council Prefect of HFC,Working hard and handing people,No,Marketing,Instagram,I believe that my life's purpose is to become ...,"Brand name, Ethical stance, Employee branding,..."
891,Nipun Samarasinghe,No,3/19/1998,22.87,Male,Yes,NSBM,"Events, Tourism and Hospitality Management",Tourism and event management,2021,...,CCHRM,Interact club organising committee member Scho...,"Rugby, Badminton",No experience,"Working together with teammates, listen to oth...",No,Marketing,Word of mouth,My purpose in life is to be an independent wom...,Unilever is one of the bigges

In [6]:
filt = (df['Gender'] == 'Male') & (df['Professional Qualification'] == 'No')

In [7]:
df.loc[filt]

,Name,Selected,Date of Birth,Age,Gender,LinkedIn Profile,University,Degree Course,Area Of Study,Graduation Year,...,Clubs & Associations,Sports,Leadership,Skills & Abilities,Working Experience,Function,Source of Information,What do you think is your life’s purpose and why do you think having a purpose is important? (100 words or less),Describe 5 things that attracted you to Unilever.,Unnamed: 22
0,Ambepitiyage Nalin Imesh De Silva,Yes,12/7/1996,24.15,Male,Yes,CA Sri Lanka,ACCA,Finance and Accountancy,2019,...,1) Interact Club of Isipathana College - Assis...,Swimming (2011/12),NaN,- Ability to handle work under pressure while ...,No,Customer Development,Word of mouth,The purpose of my life would be to take every ...,- Industry - Branding - Exposure - Recognition...,NaN
11,Siva Sharangan Srikantharaj,Yes,12/16/1996,24.13,Male,Yes,Royal Institute of Colombo,BSc. in Economics and Finance,Economics,2019,...,?Royal Institute of Colombo 1. Student Represe...,Athletics Hockey First XI Basketball Swimming,Student Representative. Head of Students Commi...,I believe that through the above mentioned ext...,No,Customer Development,Instagram,I believe my life's purpose is to achieve my l...,1) Wonderful work environment. 2) Growth oppor...,NaN
171,Don Nipuna Madushan Samaradiwakara,No,6/13/1992,28.63,Male,Yes,ACBT,"Edith Cowan University, Bachelor of Commerce B...","International Business, Management",2021,...,No,"Badminton team, Sri Dharmaloka College",• President of Sinhala literary Association of...,Currently I’m working as Business Analyst of A...,Yes,Supply Chain (Engineering Students are Preferred),Facebook,I believe my purpose in life is to help the pe...,"1. Unilever believes, “Profitable growth shoul...",NaN
253,Kavish Malinda Bandara Mapa,No,7/13/1996,24.55,Male,Yes,NSBM,Bsc(Hons) International Management and Business,Human resource management,2020,...,Didn't participate,"Badminton, Swimming, Rifle shooting",Group leader.,How to manage the team in different situations...,No,HR,Facebook,I want to be successful in life from every asp...,The Reputation The Positive word of mouth The ...,NaN
444,Shehan malaka,No,5/27/1997,23.68,Male,No,NIBM,Management,Logistic,2021,...,Prefect Youth club president Rotaryclub member,School :- volleybal football,Prefect,I learned work with a group and directing them...,No,Supply Chain (Engineering Students are Preferred),Instagram,My purpose is to live in an educated and knowl...,The working environment. The working exposure....,NaN
476,Jayaweera Arachchige Sudaraka Dananjaya Pathum,No,3/6/1999,21.90,Male,Yes,Loyola college,No any one,Sales & marketing,2022,...,No,Volleyball,Cadet Sargent Volleyball captain Class monitor,I raise many skills,Yes,Marketing,Instagram,My life's purpose it to become the best versio...,1. A well reputed company. 2. Career developme...,NaN
759,Rupasinghe Mudiyanselage Ruvinda Nilakshika Ru...,No,11/6/1997,23.24,Male,Yes,University of Kelaniya,BSc. in Management and Information Technology(...,Business Systems Engineering,2022,...,AIESEC in University of Kelaniya - Marketing C...,Athletics - running - School,Marketing Coordinator for Outgoing Global Volu...,Teamwork skills Organization skills Leadership...,Yes,Marketing,Facebook,Questions that may come up when I reflect upon...,"Fulfil consumer expectations ,Reputed company,...",NaN
893,Yuvinesh Advay Iyer,No,12/5/1999,21.16,Male,Yes,ICHEM,Bsc inorganic chemistry,Chemistry,2022,...,Lions club,Swimming,I was a secretary of lions club,"Yup, i learn to work according to time frame ...",Yes,Marketing,Word of mouth,"The purpose of my life is to be a successful, ...",1. The well-reputed name of the company 2. Qua...,NaN


In [8]:
filt = (df['Professional Qualification'] == 'No') | (df['Professional Qualification'] == 'None.')

In [9]:
df.loc[filt,['Name','Selected']]

,Name,Selected
0,Ambepitiyage Nalin Imesh De Silva,Yes
1,Dakshika Chandalee Wijesekera,Yes
6,Thilini Wasana Costa,Yes
7,Vithanage Yashodha Sammani Perera,Yes
11,Siva Sharangan Srikantharaj,Yes
171,Don Nipuna Madushan Samaradiwakara,No
228,A.M. Himasha Nethali Abeykoon,No
253,Kavish Malinda Bandara Mapa,No
310,Beligaswatta Arachchige Isuri Thakshala,No
444,Shehan malaka,No


In [14]:
df

,Name,Selected,Date of Birth,Age,Gender,LinkedIn Profile,University,Degree Course,Area Of Study,Graduation Year,...,Clubs & Associations,Sports,Leadership,Skills & Abilities,Working Experience,Function,Source of Information,What do you think is your life’s purpose and why do you think having a purpose is important? (100 words or less),Describe 5 things that attracted you to Unilever.,Unnamed: 22
0,Ambepitiyage Nalin Imesh De Silva,True,12/7/1996,24,Male,Yes,CA Sri Lanka,ACCA,Finance and Accountancy,2019,...,1) Interact Club of Isipathana College - Assis...,Swimming (2011/12),NaN,- Ability to handle work under pressure while ...,No,Customer Development,Word of mouth,The purpose of my life would be to take every ...,- Industry - Branding - Exposure - Recognition...,NaN
1,Dakshika Chandalee Wijesekera,True,8/2/1996,24,Female,Yes,University of Peradeniya,Bachelors degree in Psychology (Hons),Psychology,2021,...,"SCHOOL #Interact Club- President, Director of...","#Athletics (400m, 800m, 1500m & relays) #Gymna...",SCHOOL #School Captain & House Captain -Scho...,Always look at the bigger picture Through tra...,Yes,HR,University,My life purpose is living a successful life by...,Being a reputed and a multi national company. ...,NaN
2,Supeshala Colombathanthri,True,5/2/1997,24,Female,Yes,SLIIT,Batchelors in Business Management,Specialization in Business Analytics,2021,...,"•Gavels club , SLIIT (Editor) •AIESEC , SLIIT ...","•Tennis (National Ranker, rank 08) •Wushu (Nat...","•School Prefect (Senior, Juniorand Primary) •S...","Self Decipline it, is the main skill that I le...",Yes,Marketing,Word of mouth,"My purpose is to, ""Inspire the Vibe to, beat t...",01)Company reputation 02)Brands of the company...,NaN
3,Deshan Anselam Benhur,True,7/12/1995,26,Male,Yes,"Swinburne University of Technology, Melbourne,...",Completed my Bachelor's degree,Mechanical Engineering,2019,...,"• President, Engineers Australia, Swinburne Sa...",Basketball,School Senior Prefect University Student council,• Project Management - As the organizing chair...,Yes,Supply Chain (Engineering Students are Preferred),Facebook,My life's purpose is to genuinely serve and he...,Unilever has State of the art factories and as...,NaN
4,Masinghe Arachchige Yasaransi Pabasara Masinghe,True,2/13/1995,26,Female,Yes,University of Peradeniya,Bachelor of Business Administration,Human Resource Management,2020,...,Vice President(External Relations)- Blue Marbl...,Participated inter house hockey matches at sch...,Vice President (External Relations) Blue Marbl...,Acquiring a skill may probably be facile but p...,No,HR,Word of mouth,To be a little spark in everyone's never wande...,At the very first visit of mine to Unilever du...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889,Rathnayake Mudiyanselage Tashikala Pavithra Ra...,False,6/14/1995,26,Female,Yes,University of Sabaragamuwa,Hons in ICT,ICT,2021,...,No,"Volleyball, Netball","Prefect,president in Buddhist society,member a...",Working as a L.CPL in cadet team I was made as...,No,Customer Development,Instagram,Having a true purpose about life will only all...,1. A place where people from all cultures are ...,NaN
890,Chrishani christy,False,10/27/1999,21,Female,Yes,Bms,BSC in internation business management,Management and marketing,2021,...,I’m the secretary of the student council in Bm...,Basket ball,Secretary of student council Prefect of HFC,Working hard and handing people,No,Marketing,Instagram,I believe that my life's purpose is to become ...,"Brand name, Ethical stance, Employee branding,...",NaN
891,Nipun Samarasinghe,False,3/19/1998,23,Male,Yes,NSBM,"Events, Tourism and Hospitality Management",Tourism and event management,2021,...,Interact club organising committee member Scho...,"Rugby, Badminton",No experience,"Working together with teammates, listen to oth...",No,Marketing,Word of mouth,My purpose in life is to be an independent wom...,Unilever is one of the biggest companies 

In [20]:
df

,Name,Selected,Date of Birth,Age,Gender,LinkedIn Profile,University,Degree Course,Area Of Study,Graduation Year,...,Clubs & Associations,Sports,Leadership,Skills & Abilities,Working Experience,Function,Source of Information,purpose,unilever,Unnamed: 22
0,Ambepitiyage Nalin Imesh De Silva,True,12/7/1996,24,Male,Yes,CA Sri Lanka,ACCA,Finance and Accountancy,2019,...,1) Interact Club of Isipathana College - Assis...,Swimming (2011/12),NaN,- Ability to handle work under pressure while ...,No,Customer Development,Word of mouth,The purpose of my life would be to take every ...,- Industry - Branding - Exposure - Recognition...,NaN
1,Dakshika Chandalee Wijesekera,True,8/2/1996,24,Female,Yes,University of Peradeniya,Bachelors degree in Psychology (Hons),Psychology,2021,...,"SCHOOL #Interact Club- President, Director of...","#Athletics (400m, 800m, 1500m & relays) #Gymna...",SCHOOL #School Captain & House Captain -Scho...,Always look at the bigger picture Through tra...,Yes,HR,University,My life purpose is living a successful life by...,Being a reputed and a multi national company. ...,NaN
2,Supeshala Colombathanthri,True,5/2/1997,24,Female,Yes,SLIIT,Batchelors in Business Management,Specialization in Business Analytics,2021,...,"•Gavels club , SLIIT (Editor) •AIESEC , SLIIT ...","•Tennis (National Ranker, rank 08) •Wushu (Nat...","•School Prefect (Senior, Juniorand Primary) •S...","Self Decipline it, is the main skill that I le...",Yes,Marketing,Word of mouth,"My purpose is to, ""Inspire the Vibe to, beat t...",01)Company reputation 02)Brands of the company...,NaN
3,Deshan Anselam Benhur,True,7/12/1995,26,Male,Yes,"Swinburne University of Technology, Melbourne,...",Completed my Bachelor's degree,Mechanical Engineering,2019,...,"• President, Engineers Australia, Swinburne Sa...",Basketball,School Senior Prefect University Student council,• Project Management - As the organizing chair...,Yes,Supply Chain (Engineering Students are Preferred),Facebook,My life's purpose is to genuinely serve and he...,Unilever has State of the art factories and as...,NaN
4,Masinghe Arachchige Yasaransi Pabasara Masinghe,True,2/13/1995,26,Female,Yes,University of Peradeniya,Bachelor of Business Administration,Human Resource Management,2020,...,Vice President(External Relations)- Blue Marbl...,Participated inter house hockey matches at sch...,Vice President (External Relations) Blue Marbl...,Acquiring a skill may probably be facile but p...,No,HR,Word of mouth,To be a little spark in everyone's never wande...,At the very first visit of mine to Unilever du...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889,Rathnayake Mudiyanselage Tashikala Pavithra Ra...,False,6/14/1995,26,Female,Yes,University of Sabaragamuwa,Hons in ICT,ICT,2021,...,No,"Volleyball, Netball","Prefect,president in Buddhist society,member a...",Working as a L.CPL in cadet team I was made as...,No,Customer Development,Instagram,Having a true purpose about life will only all...,1. A place where people from all cultures are ...,NaN
890,Chrishani christy,False,10/27/1999,21,Female,Yes,Bms,BSC in internation business management,Management and marketing,2021,...,I’m the secretary of the student council in Bm...,Basket ball,Secretary of student council Prefect of HFC,Working hard and handing people,No,Marketing,Instagram,I believe that my life's purpose is to become ...,"Brand name, Ethical stance, Employee branding,...",NaN
891,Nipun Samarasinghe,False,3/19/1998,23,Male,Yes,NSBM,"Events, Tourism and Hospitality Management",Tourism and event management,2021,...,Interact club organising committee member Scho...,"Rugby, Badminton",No experience,"Working together with teammates, listen to oth...",No,Marketing,Word of mouth,My purpose in life is to be an independent wom...,Unilever is one of the biggest companies in th...,NaN
892,Shubham rajput,False,11/16/2000,20,Male,Yes,CCSU,Bachelor's of Business Administration,General BBA,2022,...,Finance club. I am ac